# Prepare auxiliary data

This notebook is used to prepare the auxiliary data.

Before processing, the data needs to be downloaded from [SASIP GitHub](https://github.com/sasip-climate/catalog-shared-data-SASIP/blob/main/outputs/OPA-neXtSIM_CREG025-ILBOXE140.md), e.g. for January 1995 [OpenDAP](https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/dodsC/meomopendap/extract/SASIP/model-outputs/OPA-neXtSIM_CREG025/OPA-neXtSIM_CREG025-ILBOXE140-S/1995/nextsim/Moorings_1995m01.nc).

The path to this data is set with the *raw_data* variable.

In addition, the auxiliary file contains data from the NSIDC regions (Meier et al., 2023) and based on the file called *NSIDC-0780_SeaIceRegions_EASE2-N3.125km_v1.0.nc*. The *region_data* specifies where this file is located.


Meier, W. N. & Stewart, J. S. (2023). Arctic and Antarctic Regional Masks for Sea Ice and Related Data Products. (NSIDC-0780, Version 1). [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. https://doi.org/10.5067/CYW3O8ZUNIWC 

In [ ]:
raw_path = "*** MISSING ***"
region_path = "*** MISSING ***" # Something like "../../data/auxiliary/NSIDC-0780_SeaIceRegions_EASE2-N3.125km_v1.0.nc"

store_path = "../data/auxiliary/ds_auxiliary.nc"

# Imports and projection definition

In [ ]:
import xarray as xr
import numpy as np
from scipy.spatial import KDTree
import cartopy.crs as ccrs

In [ ]:
src_grid = ccrs.PlateCarree()
region_grid = ccrs.LambertAzimuthalEqualArea(false_easting=0, false_northing=0., central_longitude=0., central_latitude=90.)
target_grid = ccrs.NorthPolarStereo()

# Load neXtSIM data

In [ ]:
moorings = xr.open_dataset(raw_path + "Moorings_1995m01.nc")
moorings = moorings.isel(time=0, y=slice(91, None), x=slice(8, -8), drop=True)

In [ ]:
ds_aux = moorings[["latitude", "longitude"]]
ds_aux.attrs = {}

# Estimate mask

In [ ]:
mask = 1-np.isnan(moorings["sit"]).astype(np.float32)
ds_aux["mask"] = mask

# Estimate coordinates

In [ ]:
model_coords = target_grid.transform_points(src_grid, ds_aux["longitude"].values, ds_aux["latitude"].values)[..., :2]
ds_aux["x_coord"] = (("y", "x"), model_coords[..., 0])
ds_aux["y_coord"] = (("y", "x"), model_coords[..., 1])

# Get regions

In [ ]:
ds_regions = xr.open_dataset(region_path)

In [ ]:
regions_coords = np.stack(np.meshgrid(ds_regions["x"], ds_regions["y"]), axis=-1)
regions_coords = target_grid.transform_points(region_grid, regions_coords[..., 0], regions_coords[..., 1])[..., :2]

## Use nearest neighbor

In [ ]:
tree_regions = KDTree(regions_coords.reshape(-1, 2))
_, idx = tree_regions.query(model_coords.reshape(-1, 2), k=1)

In [ ]:
regions = ds_regions["sea_ice_region"].values
regions = regions.reshape(-1)[idx.reshape(512, 512)].astype(int)

In [ ]:
regions[regions > 12] = 0
regions[(regions == 9)|(regions == 10)] = 9
regions[regions == 11] = 10
regions[regions == 12] = 11

In [ ]:
flag_meanings = ds_regions["sea_ice_region"].attrs["flag_meanings"].split(" ")
flag_meanings = flag_meanings[:13]
_ = flag_meanings.pop(10)
_ = flag_meanings.pop(9)
flag_meanings.insert(9, "baffin_bay_and_labrador_seas_and_gulf_of_st_lawrence")

## Set array

In [ ]:
ds_aux["regions"] = (("y", "x"), regions)
ds_aux["regions"] = ds_aux["regions"].assign_attrs(
    flag_values=np.arange(len(flag_meanings)),
    flag_meanings=flag_meanings,
)

# Get area approximation

In [ ]:
coords = ds_aux[["x_coord", "y_coord"]].to_array("coord_names")

In [ ]:
dx = (coords.shift(x=-1)-coords.shift(x=1)) / 2
dx = dx.where(np.isfinite(dx), coords.shift(x=-1)-coords)
dx = dx.where(np.isfinite(dx), coords-coords.shift(x=1))
dx = np.sqrt((dx**2).sum("coord_names"))

In [ ]:
dy = (coords.shift(y=-1)-coords.shift(y=1)) / 2
dy = dy.where(np.isfinite(dy), coords.shift(y=-1)-coords)
dy = dy.where(np.isfinite(dy), coords-coords.shift(y=1))
dy = np.sqrt((dy**2).sum("coord_names"))

In [ ]:
ds_aux = ds_aux.assign(
    dx=dx,
    dy=dy,
    cell_area=np.abs(dx) * np.abs(dy)
)

# Write auxiliary array

In [ ]:
ds_aux.to_netcdf(store_path, mode="w")